# Lending Club Dataset

## Feature pipeline

In [1]:
# !pip uninstall hopsworks -y

In [2]:
# !pip install hsfs-3.2.0.dev1-py3-none-any.whl 

In [24]:
get_ipython()

### Colab-Specific cell

We need to download the feature group modules (loans.py, applicants.py) in the features package (directory.

In [20]:
import os

def running_in_colab():
    return 'google.colab' in str(get_ipython())

if running_in_colab():
    print("Using colab")
    os.system('mkdir -p features')
    os.system('cd features && wget https://raw.githubusercontent.com/jimdowling/hopsworks-tutorials/loan_approval/loan_approval/features/loans.py')
    os.system('cd features && wget https://raw.githubusercontent.com/jimdowling/hopsworks-tutorials/loan_approval/loan_approval/features/applicants.py')
else:
    print("Not using colab")

Not using colab


In [3]:
import pandas as pd
import numpy as np

pd.set_option('display.float', '{:.2f}'.format)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

In [4]:
loans_df = pd.read_parquet("https://repo.hops.works/dev/jdowling/loans.parquet")
loans_df.head()

,id,loan_amnt,term,int_rate,installment,grade,sub_grade,issue_d,loan_status,purpose,title,address
0,0,10000.00,36 months,11.44,329.48,B,B4,Jan-2015,Fully Paid,vacation,Vacation,"0174 Michelle Gateway\r\nMendozaberg, OK 22690"
1,1,8000.00,36 months,11.99,265.68,B,B5,Jan-2015,Fully Paid,debt_consolidation,Debt consolidation,"1076 Carney Fort Apt. 347\r\nLoganmouth, SD 05113"
2,2,15600.00,36 months,10.49,506.97,B,B3,Jan-2015,Fully Paid,credit_card,Credit card refinancing,"87025 Mark Dale Apt. 269\r\nNew Sabrina, WV 05113"
3,3,7200.00,36 months,6.49,220.65,A,A2,Nov-2014,Fully Paid,credit_card,Credit card refinancing,"823 Reid Ford\r\nDelacruzside, MA 00813"
4,4,24375.00,60 months,17.27,609.33,C,C5,Apr-2013,Charged Off,credit_card,Credit Card Refinance,"679 Luna Roads\r\nGreggshire, VA 11650"


In [5]:
loans_df.describe()

,id,loan_amnt,int_rate,installment
count,396030.00,396030.00,396030.00,396030.00
mean,198014.50,14113.89,13.64,431.85
std,114324.16,8357.44,4.47,250.73
min,0.00,500.00,5.32,16.08
25%,99007.25,8000.00,10.49,250.33
50%,198014.50,12000.00,13.33,375.43
75%,297021.75,20000.00,16.49,567.30
max,396029.00,40000.00,30.99,1533.81


In [6]:
loans_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396030 entries, 0 to 396029
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   id           396030 non-null  int64  
 1   loan_amnt    396030 non-null  float64
 2   term         396030 non-null  object 
 3   int_rate     396030 non-null  float64
 4   installment  396030 non-null  float64
 5   grade        396030 non-null  object 
 6   sub_grade    396030 non-null  object 
 7   issue_d      396030 non-null  object 
 8   loan_status  396030 non-null  object 
 9   purpose      396030 non-null  object 
 10  title        394275 non-null  object 
 11  address      396030 non-null  object 
dtypes: float64(3), int64(1), object(8)
memory usage: 36.3+ MB


In [7]:
applicants_df = pd.read_parquet("https://repo.hops.works/dev/jdowling/applicants.parquet")
applicants_df.head()

,emp_title,emp_length,home_ownership,annual_inc,verification_status,dti,earliest_cr_line,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,id
0,Marketing,10+ years,RENT,117000.00,Not Verified,26.24,Jun-1990,16.00,0.00,36369.00,41.80,25.00,w,INDIVIDUAL,0.00,0.00,0
1,Credit analyst,4 years,MORTGAGE,65000.00,Not Verified,22.05,Jul-2004,17.00,0.00,20131.00,53.30,27.00,f,INDIVIDUAL,3.00,0.00,1
2,Statistician,< 1 year,RENT,43057.00,Source Verified,12.79,Aug-2007,13.00,0.00,11987.00,92.20,26.00,f,INDIVIDUAL,0.00,0.00,2
3,Client Advocate,6 years,RENT,54000.00,Not Verified,2.60,Sep-2006,6.00,0.00,5472.00,21.50,13.00,f,INDIVIDUAL,0.00,0.00,3
4,Destiny Management Inc.,9 years,MORTGAGE,55000.00,Verified,33.95,Mar-1999,13.00,0.00,24584.00,69.80,43.00,f,INDIVIDUAL,1.00,0.00,4


In [8]:
applicants_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396030 entries, 0 to 396029
Data columns (total 17 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   emp_title             373103 non-null  object 
 1   emp_length            377729 non-null  object 
 2   home_ownership        396030 non-null  object 
 3   annual_inc            396030 non-null  float64
 4   verification_status   396030 non-null  object 
 5   dti                   396030 non-null  float64
 6   earliest_cr_line      396030 non-null  object 
 7   open_acc              396030 non-null  float64
 8   pub_rec               396030 non-null  float64
 9   revol_bal             396030 non-null  float64
 10  revol_util            395754 non-null  float64
 11  total_acc             396030 non-null  float64
 12  initial_list_status   396030 non-null  object 
 13  application_type      396030 non-null  object 
 14  mort_acc              358235 non-null  float64
 15  

# Feature Engineering

In [9]:
from features import loans, applicants

import timeit
start = timeit.timeit()

applicants_df['home_ownership'] = applicants_df.home_ownership.apply(applicants.home_ownership)
applicants_df.home_ownership.value_counts()

end = timeit.timeit()
print(end - start)

-0.0026046810671687126


In [10]:
applicants_df.home_ownership.value_counts()

MORTGAGE    198348
RENT        159790
OWN          37746
OTHER          146
Name: home_ownership, dtype: int64

In [11]:
from features import loans, applicants

start = timeit.timeit()

applicants_df['home_ownership'] = applicants.home_ownership2(applicants_df['home_ownership'])
end = timeit.timeit()
print(end - start)


-0.00026449200231581926


In [12]:
applicants_df.home_ownership.value_counts()

MORTGAGE    198348
RENT        159790
OWN          37746
OTHER          146
Name: home_ownership, dtype: int64

In [13]:
applicants_df['earliest_cr_line'] = pd.to_datetime(applicants_df['earliest_cr_line'])
loans_df['issue_d'] = pd.to_datetime(loans_df['issue_d'])

## ✔️ `dti`, `open_acc`, `revol_bal`, `revol_util`, & `total_acc`

> - `dti`: A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income.
> - `open_acc`: The number of open credit lines in the borrower's credit file.
> - `revol_bal`: Total credit revolving balance
> - `revol_util`: Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.
> - `total_acc`: The total number of credit lines currently in the borrower's credit file

In [14]:
applicants_df['pub_rec'] = applicants_df.pub_rec.apply(applicants.pub_rec)
applicants_df['pub_rec_bankruptcies'] = \
    applicants_df.pub_rec_bankruptcies.apply(applicants.pub_rec_bankruptcies)

# 🔄 Data PreProcessing

**Section Goals:** 
> - Remove or fill any missing data. 
> - Remove unnecessary or repetitive features. 
> - Convert categorical string features to dummy variables.

Realistically there are too many unique job titles to try to convert this to a dummy variable feature. Let's remove that emp_title column.

In [15]:
applicants_df.drop('emp_title', axis=1, inplace=True)

Charge off rates are extremely similar across all employment lengths. So we are going to drop the `emp_length` column.

In [16]:
applicants_df.drop('emp_length', axis=1, inplace=True)

The title column is simply a string subcategory/description of the purpose column. So we are going to drop the title column.

In [17]:
loans_df.drop('title', axis=1, inplace=True)

### `mort_acc`

There are many ways we could deal with this missing data. We could attempt to build a simple model to fill it in, such as a linear model, we could just fill it in based on the mean of the other columns, or you could even bin the columns into categories and then set NaN as its own category. There is no 100% correct approach! 

Let's try the fillna() approach. We will group the dataframe by the total_acc and calculate the mean value for the mort_acc per total_acc entry. To get the result below:

In [18]:
total_acc_avg = applicants.mean_mort_acc(applicants_df)

/home/jdowling/Projects/hopsworks-tutorials/loan_approval/features/applicants.py:55: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  return applicants_df.groupby(by='total_acc').mean().mort_acc


In [19]:
applicants_df['mort_acc'] = applicants_df.apply(lambda x: applicants.fill_mort_acc(x['total_acc'], 
                                                        x['mort_acc'], total_acc_avg), axis=1)

KeyboardInterrupt: 

### `revol_util` & `pub_rec_bankruptcies`
These two features have missing data points, but they account for less than 0.5% of the total data. So we are going to remove the rows that are missing those values in those columns with dropna().

In [ ]:
applicants_df.dropna(inplace=True)

## 🧮 Categorical Variables

### `term`

### `grade` & `sub_grade`

We know that `grade` is just a sub feature of `sub_grade`, So we are goinig to drop it.

In [ ]:
loans_df.drop('grade', axis=1, inplace=True)

### `address`
We are going to feature engineer a zip code column from the address in the data set. Create a column called 'zip_code' that extracts the zip code from the address column.

In [ ]:
loans_df['zip_code'] = loans_df.apply(lambda x: loans.zipcode(x['address'][-5:]), axis=1)

In [ ]:
loans_df.zip_code.value_counts()

In [ ]:
loans_df.drop('address', axis=1, inplace=True)

### `issue_d` 

This is the event_time for the loan being issued

In [ ]:
loans_df.info()

### `earliest_cr_line`
This appears to be a historical time stamp feature. Extract the year from this feature using a `.apply()` function, then convert it to a numeric feature.

In [ ]:
applicants_df['earliest_cr_line_year'] = applicants_df.apply(
    lambda x: applicants.earliest_cr_line(x['earliest_cr_line']), axis=1)

In [ ]:
from great_expectations.core import ExpectationSuite, ExpectationConfiguration

expectation_suite = ExpectationSuite(
    expectation_suite_name="transaction_suite")

expectation_suite.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={
            "column":"int_rate", 
            "min_value":"-2.0",
            "max_value":"2000.0",
        }
    )
)

# TODO: Add more expectations here to improve data validation checks

## 🔖 Hopsworks Feature Store

In [ ]:
# import os
# key=""
# with open("api-key.txt", "r") as f:
#     key = f.read().rstrip()
# os.environ['HOPSWORKS_PROJECT']="loan_approval"
# # os.environ['HOPSWORKS_HOST']="staging.cloud.hopsworks.ai"
# os.environ['HOPSWORKS_HOST']="34.78.132.229"
# os.environ['HOPSWORKS_API_KEY']=key

In [ ]:
import hsfs
conn = hsfs.connection(
    '34.78.132.229',                      # Hostname of your Feature Store instance
    project='loan_approval',                       
    api_key_file="api-key.txt",
    hostname_verification=False)         # Disable for self-signed certificates
fs = conn.get_feature_store()  

In [ ]:
# import hsfs
# conn = hsfs.connection(
#     host='fdf29930-d0c6-11ed-97ad-198ac4e62551.cloud.hopsworks.ai',
#     project='loan_approval',
#     api_key_value="6WKlc22FIbvWonV4.fiqBHZj2uN3nNnPfAmwE4lCWr6AviVUa3FgBkWoghCSML0GYQMU7HyR4hQlDCSLz",
# )
# fs = conn.get_feature_store()

In [ ]:
loans_fg = fs.get_or_create_feature_group(name="loans",
                                    version=1,
                                    description="Lending Club Loans",
                                    online_enabled=True,
                                    primary_key=['id'],
                                    event_time='issue_d',
                                    expectation_suite=expectation_suite
)

In [ ]:
applicants_fg = fs.get_or_create_feature_group(name="applicants",
                                    version=1,
                                    description="Lending Club Loan Applicants",
                                    online_enabled=True,
                                    primary_key=['id'],
                                    partition_key=['earliest_cr_line_year'],
                                    event_time='earliest_cr_line'
)

In [ ]:
import time
start_time = time.time()


loans_fg.insert(loans_df, write_options={"wait_for_job" : False,
#                                         "kafka_producer_config": 
#                                          {"linger.ms": 20,
#                                           "batch.size": 1000000,
#                                           "acks": 1,
#                                          } 
                                        }
               )

#                                           "max.in.flight.requests.per.connection": 5,
#                                           "message.max.bytes": 2000000,
#                                           "batch.num.messages": 200000,
#                                           "buffer.memory": 335544320,
#                                          "queue.buffering.max.messages": 10000000,
#                                           "debug": "broker,topic,msg,queue"

print("Upload time %s seconds ---" % (time.time() - start_time))


In [ ]:
applicants_fg.insert(applicants_df, write_options={"wait_for_job" : False})

In [ ]:
metadata = pd.read_csv("https://repo.hops.works/dev/jdowling/LCDataDictionary.csv")
metadata

## Update the description of any features found in the data dictionary

Loop through the datadict. For each entry, if there is a corresponding feature in the feature group, update its description

In [ ]:
datadict=[]

# For each (name, description) pair in LCDataDictionary.csv, we try and set the feature 
# description for loans_fg and applicants_fg. If the feature doesn't exist in the feature group
# an exception is thrown, and we "do nothing"
for entry in metadata.index: 
        name = metadata['LoanStatNew'][entry]
        try:
            f = loans_fg.get_feature(name)
            loans_fg.update_feature_description(name, metadata['Description'][entry])
            print("Updating description of feature: {}".format(f.name))
        except:
            pass # do nothing

        try:
            f = applicants_fg.get_feature(name)
            applicants_fg.update_feature_description(name, metadata['Description'][entry])
            print("Updating description of feature: {}".format(f.name))
        except:
            pass # do nothing
